In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, IntegerType
from pyspark.sql.functions import from_json, col, avg, sum, window
import time

# Define the path to the jars on the EC2 instance
spark_jars_path = "/home/ec2-user/stream-processing-template/jars"  # <-- Update this path

spark = SparkSession.builder.appName("retail_pysaprk_consumer") \
    .config("spark.jars", f"{spark_jars_path}/commons-pool2-2.11.1.jar,"
            f"{spark_jars_path}/spark-sql-kafka-0-10_2.12-3.4.0.jar,"
            f"{spark_jars_path}/spark-streaming-kafka-0-10-assembly_2.12-3.4.0.jar") \
    .getOrCreate()


23/12/13 13:01:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/usr/local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [2]:
# Define the schema for our data (Column names and Datatype for each column)
schema = StructType([
    StructField("store_location", StringType(), True),
    StructField("time_of_purchase", TimestampType(), True),
    StructField("product_ID", StringType(), True),
    StructField("transaction_amount", IntegerType(), True)
])

# Stream from Kafka topic

# initalise a spark object and read the stream of data using the readStream method
# set the format of the streaming source to kafka
# set the kafka topic to retail_transactions (topic is a feed name to which messages are published)
# load the data and store it in the df variable
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092") \
    .option("subscribe", "retail_transactions") \
    .load()

In [3]:
# Extract and parse the JSON data - convert the json data to strings and add it to a new column 'data'

# selectExpr allows you to select and transform columns using SQL
# CAST(value AS STRING) - SQL expression that set the datatype for the selected column as a string
# withColumn - Pyspark method used to add a or replace a column to the DF, in this case creating a new column named 'data'
# the from_json function takes the json data from the value column of the stream and inserts into the new column
transactions = (df.selectExpr("CAST(value AS STRING)")
                .withColumn("data", from_json(col("value"), schema))
                .select("data.*"))


# write the transactions DF to an in memory table called temporary_view_two

# writeStream - write the streaming data to memory, whereas readStream reads streaming data
# set the format to memory
# give the query a name using queryName
# start the streaming query
query = transactions.writeStream \
.format("memory") \
.queryName("temporary_view_two") \
.start()

query.awaitTermination(180)

23/12/13 13:01:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-89651826-c9c9-4627-9a2d-1898c24a8485. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/13 13:01:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/12/13 13:01:39 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


False

In [4]:
processed_data = spark.sql("SELECT * FROM temporary_view_two")

# Now you can perform aggregations or other transformations on `processed_data`

In [5]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

transactions = processed_data.show()

+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|       Phoenix|2023-12-13 13:01:42|    P00080|               513|
|   San Antonio|2023-12-13 13:01:44|    P00077|               470|
|       Chicago|2023-12-13 13:01:47|    P00026|               436|
|       Chicago|2023-12-13 13:01:49|    P00062|                28|
|  Philadelphia|2023-12-13 13:01:52|    P00093|               537|
|   San Antonio|2023-12-13 13:01:54|    P00097|                 9|
|      New York|2023-12-13 13:01:56|    P00051|               415|
|       Phoenix|2023-12-13 13:01:58|    P00054|               998|
|       Chicago|2023-12-13 13:02:00|    P00016|                51|
|       Houston|2023-12-13 13:02:03|    P00065|               789|
|   Los Angeles|2023-12-13 13:02:06|    P00064|               140|
|       Phoenix|2023-12-13 13:02:08|    P00028|               

Number of transactions per minute 

In [23]:
num_trans_per_min = processed_data.groupBy(window("time_of_purchase", "1 minute")).count()
num_trans_per_min = num_trans_per_min.orderBy(col('window').asc())
num_trans_per_min.show()

+--------------------+-----+
|              window|count|
+--------------------+-----+
|{2023-12-13 13:01...|    8|
|{2023-12-13 13:02...|   30|
|{2023-12-13 13:03...|   29|
|{2023-12-13 13:04...|   30|
|{2023-12-13 13:05...|   30|
|{2023-12-13 13:06...|   30|
|{2023-12-13 13:07...|   33|
|{2023-12-13 13:08...|   31|
|{2023-12-13 13:09...|   30|
|{2023-12-13 13:10...|   27|
|{2023-12-13 13:11...|   30|
|{2023-12-13 13:12...|   28|
|{2023-12-13 13:13...|   30|
|{2023-12-13 13:14...|   31|
|{2023-12-13 13:15...|   30|
|{2023-12-13 13:16...|   30|
|{2023-12-13 13:17...|   28|
|{2023-12-13 13:18...|   29|
|{2023-12-13 13:19...|   29|
|{2023-12-13 13:20...|   30|
+--------------------+-----+
only showing top 20 rows



Total of all transactions per minute

In [24]:
total_trans_amount_per_min = processed_data.groupBy(window("time_of_purchase", "1 minute")).sum("transaction_amount")
total_trans_amount_per_min = total_trans_amount_per_min.orderBy(col('window').asc())
total_trans_amount_per_min.show()


+--------------------+-----------------------+
|              window|sum(transaction_amount)|
+--------------------+-----------------------+
|{2023-12-13 13:01...|                   3406|
|{2023-12-13 13:02...|                  11386|
|{2023-12-13 13:03...|                  15193|
|{2023-12-13 13:04...|                  10881|
|{2023-12-13 13:05...|                  11320|
|{2023-12-13 13:06...|                  10128|
|{2023-12-13 13:07...|                  14988|
|{2023-12-13 13:08...|                  12621|
|{2023-12-13 13:09...|                  15401|
|{2023-12-13 13:10...|                  12549|
|{2023-12-13 13:11...|                  12769|
|{2023-12-13 13:12...|                  10432|
|{2023-12-13 13:13...|                  15261|
|{2023-12-13 13:14...|                  16939|
|{2023-12-13 13:15...|                  15157|
|{2023-12-13 13:16...|                  12890|
|{2023-12-13 13:17...|                  14218|
|{2023-12-13 13:18...|                  15278|
|{2023-12-13 

Top N Products by Number of tranactions

In [30]:
top_products = processed_data.groupBy(col('product_Id')).count()
top_products = top_products.orderBy(col('count').desc())
top_products.show()

+----------+-----+
|product_Id|count|
+----------+-----+
|    P00098|   18|
|    P00019|   17|
|    P00004|   14|
|    P00037|   13|
|    P00047|   13|
|    P00001|   13|
|    P00003|   12|
|    P00078|   11|
|    P00082|   11|
|    P00086|   11|
|    P00063|   11|
|    P00094|   11|
|    P00065|   10|
|    P00075|   10|
|    P00023|   10|
|    P00070|   10|
|    P00091|   10|
|    P00089|   10|
|    P00069|   10|
|    P00054|    9|
+----------+-----+
only showing top 20 rows



Top products by sales amount

In [36]:
top_prod_by_sales_amount = processed_data.groupBy(col('product_Id')).sum('transaction_amount')
top_prod_by_sales_amount = top_prod_by_sales_amount.orderBy(col('sum(transaction_amount)').desc())
top_prod_by_sales_amount.show()

+----------+-----------------------+
|product_Id|sum(transaction_amount)|
+----------+-----------------------+
|    P00019|                   9153|
|    P00086|                   7858|
|    P00098|                   7836|
|    P00047|                   7239|
|    P00050|                   6410|
|    P00096|                   6248|
|    P00018|                   5851|
|    P00075|                   5799|
|    P00063|                   5774|
|    P00070|                   5581|
|    P00035|                   5490|
|    P00065|                   5485|
|    P00001|                   5406|
|    P00064|                   5384|
|    P00023|                   5361|
|    P00097|                   5309|
|    P00082|                   5191|
|    P00081|                   5164|
|    P00069|                   4972|
|    P00071|                   4926|
+----------+-----------------------+
only showing top 20 rows



Anomoly Detection 

- Identify transactions with amounts significantly different from the average.


Get the average transaction ammount

In [43]:
avg_trans = processed_data.agg(avg(col('transaction_amount')).alias('avg_transaction_amount')).collect()[0]['avg_transaction_amount']
avg_trans

453.2670401493931

Filter for transactions with a transaction amount 50% larger than the average transaction amount

In [45]:
anomoly_detect = processed_data.filter(col('transaction_amount') > avg_trans * 1.5)
anomoly_detect.show()

+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|       Phoenix|2023-12-13 13:01:58|    P00054|               998|
|       Houston|2023-12-13 13:02:03|    P00065|               789|
|       Chicago|2023-12-13 13:02:17|    P00042|               797|
|   San Antonio|2023-12-13 13:02:25|    P00035|               765|
|   Los Angeles|2023-12-13 13:02:27|    P00050|               719|
|  Philadelphia|2023-12-13 13:02:31|    P00024|               722|
|   San Antonio|2023-12-13 13:02:33|    P00096|               910|
|       Phoenix|2023-12-13 13:02:37|    P00075|               941|
|       Houston|2023-12-13 13:02:50|    P00060|               730|
|   Los Angeles|2023-12-13 13:03:01|    P00086|               934|
|       Chicago|2023-12-13 13:03:08|    P00100|               861|
|      New York|2023-12-13 13:03:14|    P00033|               

Filter all transactions above and below a certain amount

In [52]:
fitlered_trans = processed_data.filter((col('transaction_amount') < 800) & (col('transaction_amount') > 750))
fitlered_trans.show()

+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|       Houston|2023-12-13 13:02:03|    P00065|               789|
|       Chicago|2023-12-13 13:02:17|    P00042|               797|
|   San Antonio|2023-12-13 13:02:25|    P00035|               765|
|   San Antonio|2023-12-13 13:04:11|    P00050|               786|
|   Los Angeles|2023-12-13 13:05:22|    P00080|               755|
|   San Antonio|2023-12-13 13:06:14|    P00070|               786|
|       Phoenix|2023-12-13 13:06:25|    P00019|               773|
|  Philadelphia|2023-12-13 13:07:00|    P00086|               783|
|       Phoenix|2023-12-13 13:07:02|    P00062|               751|
|  Philadelphia|2023-12-13 13:07:31|    P00027|               796|
|       Chicago|2023-12-13 13:07:45|    P00067|               796|
|       Chicago|2023-12-13 13:08:56|    P00018|               